In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # used for splitting training and testing data

torch.manual_seed(1)

In [223]:
plt.figure(figsize=(8,5))

# time steps per batch of data
seq_length = 100

time_steps = np.linspace(0, np.pi, seq_length + 1)

data = pd.read_csv('SectionData.csv')

x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

<Figure size 576x360 with 0 Axes>

In [248]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, ouput_size, n_layers, hidden_size, drop_prob):
        
        super(NeuralNetwork, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.drop_prob = drop_prob
                
        self.lstm = nn.LSTM(input_size, hidden_size[0], n_layers, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        # First fully connected layer
        self.fc1 = nn.Linear(hidden_size[0], hidden_size[1])
        # Second fully connected layer
        self.fc2 = nn.Linear(hidden_size[1], hidden_size[2])
        
        # Softmax activation
        self.softmax = nn.Softmax()
        
    def forward(self, x, hidden):
        # x(batch_size, seq_length, input_size)
        # hidden(n_layers, batch_size, hidden_dim)
        # r_out(batch_size, time_step, hidden_size)
        
        # Use x.view(seq_len, batch, input_size)
        # And this order corresponde to:
        # seq_len = time dimension (how many time steps do you have)
        # batch = mini-batch dimension (how much do you split ur data)
        # input_size = feature dimension (so how many features do you have)
        
        x = x.view(1, x.size(0), self.input_size)
        
        if hidden is not None:
            hidden = hidden.double()
            
        print(hidden)
        
        # get LSTM outputs
        lstm_out, hidden = self.lstm(x, hidden)
        
        # shape output to be (batch_size*seq_length, hidden_dim)
        lstm_out = lstm_out.view(-1, self.hidden_dim)
        
        # dropout layer
        out = self.dropout(lstm_out)
        
        # get output of first fully connected layer
        out1 = self.fc1(lstm_out)
        
        # get output of second fully connected layer
        out2 = self.fc2(out1)
        
        # softmax activation function
        out2 = self.softmax(out2)
        
        # TODO: might need to reshape the out2
        
        return output, hidden
    
    def init_hidden(self, batch_size):
        '''initialize hidden states'''
        # initialized initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.paramaters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_size).zero_(),
                 weight.new(self.n_layers, batch_size, self.hidden_size).zero_())
        
        return hidden

In [249]:
def preprocess(df):

    # drop ecgNum row
    df = df.drop(["ecgNum"], axis=1)
    # Classify the dependent and independent variables
    X = df.iloc[:, :-1].values
    Y = df.iloc[:, -1].values
    # split the data into train, validate, test
    X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    X_train, X_val, Y_train, Y_val = train_test_split(X_temp, Y_temp, test_size=0.2, random_state=0)
    
    return torch.from_numpy(X_train), torch.from_numpy(X_val), torch.from_numpy(X_test), Y_train, Y_val, Y_test

In [250]:
preprocess(data)

(tensor([[-0.0073, -0.0095, -0.0112,  ...,  0.0055, -0.0147, -0.0244],
         [-0.0724, -0.0792, -0.0600,  ..., -0.0284, -0.0487, -0.0392],
         [ 0.0239,  0.0090,  0.0127,  ...,  0.0102,  0.0075,  0.0116],
         ...,
         [ 0.0303,  0.0208,  0.0186,  ...,  0.0457,  0.0438,  0.0378],
         [-0.0068, -0.0079, -0.0086,  ..., -0.0128, -0.0096, -0.0071],
         [-0.0615, -0.0699, -0.0729,  ..., -0.0334, -0.0324, -0.0403]],
        dtype=torch.float64),
 tensor([[ 0.1049,  0.0905,  0.1028,  ...,  0.0754,  0.0830,  0.0788],
         [ 0.0200,  0.0152,  0.0201,  ...,  0.0371,  0.0169,  0.0145],
         [-0.0060,  0.0056, -0.0003,  ..., -0.0155, -0.0091, -0.0235],
         ...,
         [-0.0307, -0.0333, -0.0472,  ..., -0.0044, -0.0043, -0.0351],
         [-0.0370, -0.0415, -0.0533,  ..., -0.0324, -0.0406, -0.0476],
         [-0.0197, -0.0220, -0.0141,  ..., -0.0189, -0.0068,  0.0018]],
        dtype=torch.float64),
 tensor([[ 0.0951,  0.0873,  0.0820,  ...,  0.0874,  0.082

In [251]:
# size of the input at each time step
# TODO: handle input sizes
input_size = 100
# size of the hidden state and cell state at each time step
hidden_size = [64, 32, 6]
# TODO: ouptut sizes
output_size = [8, 4] # remove l8R
# number of LSTM layers
n_layers = 1
# dropout probability
drop_prob = 0.5

# instantiate the NN
neuralNet = NeuralNetwork(input_size, output_size, n_layers, hidden_size, drop_prob)
print(neuralNet)

NeuralNetwork(
  (lstm): LSTM(100, 64, batch_first=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=6, bias=True)
  (softmax): Softmax(dim=None)
)


In [252]:
# Mean Squared Error and Adam Optimizer with a learning rate of 0.01
# TODO: play with LR
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(neuralNet.parameters(), lr=0.01)

In [266]:
# train the NN
def train(neuralNet, n_steps, print_every):
    # initialize the hidden state
    hidden = torch.from_numpy(np.zeros((100,1,1,64))).double()
    
    X_train, X_val, X_test, Y_train, Y_val, Y_test = preprocess(data)
    
    for batch_i, step in enumerate(range(n_steps)):
        # zero accumulated gradients
        neuralNet.zero_grad()
        
        # output from the neural net
        prediction, hidden = neuralNet(X_train, hidden)
        
        hidden = hidden.data
        
        # calculate the loss
        loss = criterion(prediction, Y_train)
        # zero gradients
        optimizer.zero_grad()
        # backpropogate and update weights
        loss.backward()
        optimizer.step()
        
        # display loss and predictions
        if batch_i%print_every == 0:
            print('Loss ', loss.item())
            plt.plot(time_steps[1:], x, 'r.') #input
            plt.plot(time_steps[1:], predction.data.numpy().flatten(), 'b.') #predictions
            
    return neuralNet

In [267]:
n_steps = 100
print_every = 45

trained_NN = train(neuralNet, n_steps, print_every)

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.]]]], dtype=torch.float64)


TypeError: lstm() received an invalid combination of arguments - got (Tensor, Tensor, list, bool, int, float, bool, bool, bool), but expected one of:
 * (Tensor data, Tensor batch_sizes, tuple of Tensors hx, tuple of Tensors params, bool has_biases, int num_layers, float dropout, bool train, bool bidirectional)
      didn't match because some of the arguments have invalid types: ([32;1mTensor[0m, [32;1mTensor[0m, [31;1mlist[0m, [31;1mbool[0m, [31;1mint[0m, [31;1mfloat[0m, [31;1mbool[0m, [32;1mbool[0m, [32;1mbool[0m)
 * (Tensor input, tuple of Tensors hx, tuple of Tensors params, bool has_biases, int num_layers, float dropout, bool train, bool bidirectional, bool batch_first)
      didn't match because some of the arguments have invalid types: ([32;1mTensor[0m, [31;1mTensor[0m, [31;1mlist[0m, [32;1mbool[0m, [32;1mint[0m, [32;1mfloat[0m, [32;1mbool[0m, [32;1mbool[0m, [32;1mbool[0m)
